In [4]:
import pandas as pd
import csv
from Plan import CreateSamples_renewed
import importlib

In [5]:
importlib.reload(CreateSamples_renewed)

<module 'Plan.CreateSamples_renewed' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Plan\\CreateSamples_renewed.py'>

In [6]:
path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\New_Testing_Protocol_Generalized.csv"
chem_path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Chemical Database.csv"

In [7]:
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)

In [24]:
df = CreateSamples_renewed.concentration_from_linspace(plan, unity_filter=True)
df

,dppc concentration wtf,dspepeg2000 concentration wtf,pfh concentration wtf,ethanol concentration wtf,water concentration wtf
0,0.000075,0.000035,0.00,0.000000,0.999890
1,0.000075,0.000035,0.00,0.036842,0.963048
2,0.000075,0.000035,0.00,0.073684,0.926206
3,0.000075,0.000035,0.00,0.110526,0.889364
4,0.000075,0.000035,0.00,0.147368,0.852522
...,...,...,...,...,...
495,0.000075,0.000035,0.01,0.552632,0.437258
496,0.000075,0.000035,0.01,0.589474,0.400416
497,0.000075,0.000035,0.01,0.626316,0.363574
498,0.000075,0.000035,0.01,0.663158,0.326732


In [9]:
component_units = plan['Component Concentration Units']
total_sample_amount_unit = plan['Sample Unit']
total_sample_amount = plan['Sample Amount']
component_units

['wtf', 'wtf', 'wtf', 'wtf', 'wtf']

In [10]:
for col, unit in zip(component_units,df):
    if total_sample_amount_unit == 'g':
        if unit == 'wtf':
            mass = total_sample_amount*df[col] # you could raise a notification on what info is needed for stock with wtf
        if unit == 'mgpermL'
    if total_sample_amount_unit =='mL':
        if unit == 'mgpermL':
            # so when being labeled in the dataframe just make it clear that it is a mass of the component
        if unit == 'volf'
            # make it clear it is a volume

SyntaxError: invalid syntax (<ipython-input-10-6caff74dd94d>, line 5)

In [11]:
# need to split up based on what it will return and not what it starts with

In [19]:
def identify_unit(string):
    list_units = ['wtf','volf','molf','mgpermL','molarity']
    for unit in list_units:
        if unit in string:
            return unit

In [23]:
for column_name in df:
    component_unit = identify_unit(column_name)
    if component_unit == 'wtf' or 'mgpermL' or 'molarity': # these are the unit that lead to mass outcomes
        component_masses = 
        
    

mom
mom
mom
mom
mom


In [ ]:
def determine_component_mass(component_unit, component_values, sample_amount_unit, sample_amount):
    if sample_amount_unit == 'g' and component_unit = 'wtf':
        component_masses = sample_amount*component_values
        
    if sample_amount_unit == 'mL' and component_unit = 'mgpermL':
        component_masses = sample_amount*component_values/1000 # for now default mass = g, volume = mL

    if sample_amount_unit == 'mL' and component_unit = 'molarity':
        molecular_weight = 
        component_masses = sample_amount*component_values/1000
        
    return component_masses
